In [1]:
import os
import cv2 
import tifffile
import statistics
import numpy as np

from tqdm import tqdm
from glob import glob
from scipy import ndimage as ndi
from skimage.io import imsave
from skimage import (
    measure,
    morphology,
    filters,
    exposure,
    img_as_ubyte,
)

from commons import rescale_arr

/home/mpgxc/anaconda3/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


## Funções Auxiliares

In [2]:
def read_image_frames_seq(path):
    def parser_image_name(image_name):

        *_, name = image_name.split("/")
        name, *_ = name.split(".")

        return int(name)
    
    arr = []
    
    for index in glob(path + "/*"):
        try:
            image_name = parser_image_name(index)
            
            arr.append(image_name)
            
        except:
            continue
    
    image_path_sorted = sorted(arr)

    image_unique_name = lambda x: f"{path}/{x}.tif"

    return list(map(image_unique_name, image_path_sorted))


def build_volume_from_directory(arr_paths, with_rgb=False):
    """
        Ler todas as imagens do diretório e cria um bloco de imagens
    """
    if with_rgb:
        return np.asarray([tifffile.imread(img) for img in arr_paths])

    return np.asarray([tifffile.imread(img)[:, :, 0] for img in arr_paths])

def find_bighest_cluster_area(clusters):
    """
        Essa função deve receber uma imagem segmentada (Clusters)
        Retorna a área do maior cluster
    """
    regions = measure.regionprops(clusters)

    def area(item):
        return item.area

    return max(map(area, regions))


def find_best_larger_cluster(image_mask):

    clusters = image_mask.copy()

    """
        Verificação para saber se a mascara está correta c:
    """

    if statistics.mode(clusters.flatten()):
        clusters = np.invert(clusters)

    clusters = measure.label(clusters, background=0)

    cluster_size = find_bighest_cluster_area(clusters)

    return morphology.remove_small_objects(
        clusters.astype(dtype=bool),
        min_size=(cluster_size - 1), 
        connectivity=8
    )

def binarize_image(arr):
    return arr > filters.threshold_triangle(arr)

def check_colision_border(mask):
    """
        Pós processamento
    """
    x, *_ = mask.shape

    left = mask[:1, ].flatten()
    right = mask[x - 1: x, ].flatten()
    top = mask[:, : 1].flatten()
    bottom = mask[:, x - 1: x].flatten()

    borders_flatten = [left, right, top, bottom]
    if np.concatenate(borders_flatten).sum():
        return True

    return False

## Função Principal

In [3]:
def make_mask(image, background, fn_equalize=None):
    def apply_thresh(image, block_size=1, thresh_type=199):
        return cv2.adaptiveThreshold(image, 
                                 255,
                                 cv2.ADAPTIVE_THRESH_MEAN_C,
                                 cv2.THRESH_BINARY, 
                                 thresh_type, 
                                 block_size).astype(bool)
    mask = apply_thresh(image)
    
    if statistics.mode(mask.flatten()):
        mask = np.invert(mask)

    if not fn_equalize:
        mask_bin = binarize_image(
            np.subtract(background,
                        image)
        )
    else:
        mask_bin = binarize_image(
            np.subtract(fn_equalize(background),
                        fn_equalize(image))
        )

    if statistics.mode(mask_bin.flatten()):
        mask_bin = np.invert(mask_bin)

    mask = find_best_larger_cluster(mask)
    
    mask = find_best_larger_cluster(mask_bin + mask)
    
    if check_colision_border(mask):
        for block in range(1, 9):

            mask = mask = apply_thresh(image, block)
            mask = find_best_larger_cluster(mask)
            
            if not check_colision_border(mask): break

    if statistics.mode(mask.flatten()):
        mask = np.invert(mask)
    
    mask = morphology.closing(mask, morphology.disk(3))

    mask = ndi.binary_fill_holes(mask)

    mask = filters.gaussian(mask, sigma=1.5)
    
    return binarize_image(mask)

In [4]:
def make_mask(image, background, fn_equalize=None):
    def apply_thresh(image, block_size=1, thresh_type=199):
        return cv2.adaptiveThreshold(image, 
                                 255,
                                 cv2.ADAPTIVE_THRESH_MEAN_C,
                                 cv2.THRESH_BINARY, 
                                 thresh_type, 
                                 block_size).astype(bool)
    mask = apply_thresh(image)
    
    if statistics.mode(mask.flatten()):
        mask = np.invert(mask)
    
    if check_colision_border(mask):
        for block in range(1, 9):

            mask = mask = apply_thresh(image, block)
            mask = find_best_larger_cluster(mask)
            
            if not check_colision_border(mask): break

    if statistics.mode(mask.flatten()):
        mask = np.invert(mask)
    
    mask = morphology.closing(mask, morphology.disk(3))

    mask = ndi.binary_fill_holes(mask)

    mask = filters.gaussian(mask, sigma=1.5)
    
    return binarize_image(mask)

In [5]:
out_put_folder = 'masks'

for path in glob("images_masks_cropped/images/*"):

    seq_frames = read_image_frames_seq(path)
    seq_frames_volume = build_volume_from_directory(seq_frames, with_rgb=True)
    background = rescale_arr(np.median(seq_frames_volume, axis=0))

    image_name = path.split('/')[2]
    
    try:
        os.makedirs(f'{out_put_folder}/{image_name}')
    except: pass
    
    for index, frame in enumerate(tqdm(seq_frames_volume)):
        
        try:
            
            mask = make_mask(frame, background, exposure.equalize_hist)
            
            if check_colision_border(mask): continue
                
            imsave(f"{out_put_folder}/{image_name}/{index}.tif", img_as_ubyte(mask))
            
        except Exception as e: continue

100%|██████████| 292/292 [00:09<00:00, 29.57it/s]
